## Notebook to bulk download Planet imagery through the API
Rainey Aberle, Fall 2021

Modified from [Planet Developers API Tutorial](https://developers.planet.com/docs/apis/data/) and Planet Labs GitHub Repository: [planetlabs/notebooks/jupyter-notebooks/orders/](https://github.com/planetlabs/notebooks/tree/master/jupyter-notebooks/orders)


__To-Do:__
- View image footprints before downloading. Will need to download only the metadata first, then plot the coordinates over a regional map. 
- Implement percent AOI coverage filter for image search

### Import necessary packages

In [1]:
import os
import json
import requests
import time
import geopandas as gpd
from pathlib import Path
import rasterio as rio
import numpy as np
from rasterio.plot import show
from requests.auth import HTTPBasicAuth

In [2]:
# If you don't have one of these packages installed, install it here, then run the previous cell again.
# !conda install -c conda-forge geopandas

### Install Planet API Client

This will allow you to interact with the Planet API through this notebook. Refer to the __[Planet API documentation](https://developers.planet.com/docs/apis/data/)__ for more info. 

In [3]:
!pip3 install planet

### Define filters for image search
#### _Modify these sections_

In [4]:
# ----------Area of Interest (AOI)----------
#### OPTION 1: Use geojson.io to create a polygon
#    - Type geojson.io into a new browser page
#    - Draw a RECTANGULAR polygon over your AOI  
#    - Copy only the geometry below
# AOI_box = {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [
#               -148.96774291992188,
#               60.366355109034046
#             ],
#             [
#               -148.84963989257812,
#               60.366355109034046
#             ],
#             [
#               -148.84963989257812,
#               60.440285056170616
#             ],
#             [
#               -148.96774291992188,
#               60.440285056170616
#             ],
#             [
#               -148.96774291992188,
#               60.366355109034046
#             ]
#           ]
#         ]
#       }
# AOI_box

In [5]:
#### OPTION 2: Import an existing shapefile

# Name of your file
# If your shapefile is not currently in this directory, you need to include the full file path in 'file_name' below
file_name = 'inputs/wolverineGlacier.shp'

# Read in the shapefile
AOI = gpd.read_file(file_name)

# File extension index (we don't want the .shp extension in the next line)
i = file_name.index('.shp')

# Convert to geojson
AOI.to_file(file_name[0:i]+'.geojson', driver='GeoJSON')

# Adjust AOI polygon to a rectangular shape 
# Planet only excepts a bounding BOX as a spatial filter, 
# so we need to convert our AOI to a box (if it is not already). 
AOI_box = {"type": "Polygon",
           "coordinates": [[
               [AOI.bounds.minx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.miny[0]]
           ]]
          }
AOI_box

{'type': 'Polygon',
 'coordinates': [[[-148.97493073077095, 60.35056373052069],
   [-148.83419955670448, 60.35056373052069],
   [-148.83419955670448, 60.45403189448731],
   [-148.97493073077095, 60.45403189448731],
   [-148.97493073077095, 60.35056373052069]]]}

In [6]:
# ----------AOI clipping----------
# Would you like to clip images to the AOI (True/False)?
# This greatly speeds up the ordering and downloading process.
clip_AOI = True

# ----------Date Range----------
# Format: 'YYYY-MM-DD'
start_date = "2021-04-20"
end_date = "2021-08-25"

# ----------Cloud Filter----------
# Format: decimal (e.g., 50% max cloud cover = 0.5)
max_cloud_cover = 0.2

# ----------Item Type----------
# See here for possible image ("item") types:
# https://developers.planet.com/docs/apis/data/items-assets/
item_type = "PSScene4Band"

# ----------Asset Type----------
# Each Item Type has a number of asset types to choose from.
# Use the Item Type link above click on your Item Type to view the available Asset Types
asset_type = "analytic_sr"

# ----------Planet API Key----------
# Find your API key on your Planet Account: account.planet.com
# My Settings > API Key
API_key = '21d92c1c372146c089f0182295d3b028'

# ----------Output folder----------
# AKA, where you want your images to be downloaded in your directory
out_folder = '/Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/'

---------------------
### Authentication via basic HTTP

In [7]:
# set API key as environment variable
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

### Compile filters and use Quick Search to grab image IDs

In [8]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": AOI_box
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": start_date + "T00:00:00.000Z",
    "lte": end_date + "T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": max_cloud_cover
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

# define the clip tool
clip = {
    "clip": {
        "aoi": AOI_box
    }
}

# API request object
QS_request = {
  #"interval":"day",
  "item_types": [item_type], 
  "asset_types": [asset_type],
  "filter":combined_filter
}

# fire off the POST request
QS_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=QS_request)

# Print resulting image IDs
im_ids = [feature['id'] for feature in QS_result.json()['features']]
print(im_ids)

['20210821_204640_1013', '20210821_201830_73_2428', '20210821_201828_26_2428', '20210818_204911_1014', '20210818_204909_1014', '20210818_164256_104e', '20210818_204910_1014', '20210818_164255_104e', '20210816_202318_59_242a', '20210816_202316_11_242a', '20210815_202832_67_225a', '20210815_202830_37_225a', '20210815_202055_60_2459', '20210815_202053_12_2459', '20210815_202058_05_2429', '20210815_202055_57_2429', '20210815_204455_1010', '20210815_204454_1010', '20210815_204453_1010', '20210813_204649_1012', '20210801_210822_19_240c', '20210801_210819_89_240c', '20210802_213636_22_1057', '20210802_213634_72_1057', '20210802_203626_91_225a', '20210802_203624_62_225a', '20210802_204543_1012', '20210802_204542_1012', '20210802_204541_1012', '20210801_202256_48_2456', '20210801_202254_00_2456', '20210801_193330_13_106d', '20210801_193328_61_106d', '20210801_193327_08_106d', '20210801_164622_104a', '20210801_164620_104a', '20210801_204541_1003', '20210801_164621_104a', '20210731_204804_0f17', 

### Place Order

In [10]:
# set content type to json
headers = {'content-type': 'application/json'}

# create a request object
# (clip images if clip==True)
if clip_AOI:
    request = {  
       "name":"simple order",
       "products":[
          {
              "item_ids": im_ids,
              "item_type": item_type,
              "product_bundle": asset_type
          }
       ],
        "tools": [clip]
    }
else:
    request = {  
       "name":"simple order",
       "products":[
          {
              "item_ids": im_ids,
              "item_type": item_type,
              "product_bundle": asset_type
          }
       ],
    }

# define function to place order
def place_order(search_request, auth):
    response = requests.post(orders_url, data=json.dumps(search_request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

# place order
order_url = place_order(request, auth)

<Response [202]>
04fe35bd-821f-4bed-a907-a637d3f0cb0d


### Poll for Order Success
- This section outputs the status of the order every ~10 sec. This will take a few minutes... 
- Wait until it outputs `success` to proceed to the next section. It will stop after 30 loops, so try proceeding to the next section if it finishes running and does not output `success`.
- If you are ordering a LOT of images, consider narrowing your date range to download less images at a time. 

In [13]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(order_url, auth)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running


### View Results

In [14]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

[r['name'] for r in results]

['04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_metadata.json',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_SR_clip.tif',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_metadata_clip.xml',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_DN_udm_clip.tif',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_165654_1050_3B_AnalyticMS_SR_clip.tif',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_165654_1050_3B_AnalyticMS_metadata_clip.xml',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_165654_1050_metadata.json',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_165654_1050_3B_AnalyticMS_DN_udm_clip.tif',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_203626_91_225a_3B_AnalyticMS_SR_clip.tif',
 '04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_203626_91_225a_3B_A

### Download each asset individually

In [15]:
# define function to download results
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = Path(os.path.join(out_folder,name)) #pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
# download images!
download_results(results)

636 items to download
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_SR_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_metadata_clip.xml
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_213634_72_1057_3B_AnalyticMS_DN_udm_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/i

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210614_164759_0f49_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210614_164759_0f49_3B_AnalyticMS_metadata_clip.xml
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210731_201800_68_2212_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210731_201800_68_2212_3B_AnalyticMS_SR_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210731_201800_68_2212_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210731_201800_68_2212_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210731_201800_68_2212_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210718_213547_71_1066_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210718_213547_71_1066_3B_AnalyticMS_metadata_clip.xml
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210718_213547_71_1066_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210718_213547_71_1066_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210507_170008_1054_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210507_170008_1054_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210507_170008_1054_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Ba

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210501_211757_67_227a_3B_AnalyticMS_DN_udm_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210501_211757_67_227a_3B_AnalyticMS_DN_udm_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210501_211757_67_227a_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210501_211757_67_227a_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210501_211757_67_227a_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210501_211757_67_227a_3B_AnalyticMS_metadata_clip.xml
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210613_193952_72_106c_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210426_203130_64_2251_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210426_203130_64_2251_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210426_203130_64_2251_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210426_203130_64_2251_3B_AnalyticMS_SR_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210426_203130_64_2251_3B_AnalyticMS_DN_udm_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210426_203130_64_2251_3B_AnalyticMS_DN_udm_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210426_203130_64_2251_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed

...

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_211428_03_2406_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_211428_03_2406_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_211428_03_2406_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_211428_03_2406_3B_AnalyticMS_SR_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_211428_03_2406_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_211428_03_2406_3B_AnalyticMS_metadata_clip.xml
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210425_211428_03_2406_3B_AnalyticMS_DN_udm_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4b

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210715_204552_1035_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210715_204552_1035_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210611_165002_104b_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210611_165002_104b_3B_AnalyticMS_SR_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210611_165002_104b_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210611_165002_104b_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210611_165002_104b_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210611_165002_104b_

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210423_170031_1053_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210423_170031_1053_3B_AnalyticMS_SR_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210423_170031_1053_3B_AnalyticMS_DN_udm_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210423_170031_1053_3B_AnalyticMS_DN_udm_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210815_204454_1010_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210815_204454_1010_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210815_204454_1010_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_204542_1012_3B_AnalyticMS_DN_udm_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_204542_1012_3B_AnalyticMS_DN_udm_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_204542_1012_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_204542_1012_metadata.json
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_204542_1012_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210802_204542_1012_3B_AnalyticMS_metadata_clip.xml
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210821_201830_73_2428_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-

downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210524_165519_104b_3B_AnalyticMS_SR_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210524_165519_104b_3B_AnalyticMS_SR_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210524_165519_104b_3B_AnalyticMS_metadata_clip.xml to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210524_165519_104b_3B_AnalyticMS_metadata_clip.xml
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210524_165519_104b_3B_AnalyticMS_DN_udm_clip.tif to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210524_165519_104b_3B_AnalyticMS_DN_udm_clip.tif
downloading 04fe35bd-821f-4bed-a907-a637d3f0cb0d/PSScene4Band/20210524_165519_104b_metadata.json to /Users/raineyaberle/Desktop/Research/PhD/Wolverine/images/04fe35bd-821f-4bed-a9

### Visualize downloaded images

In [ ]:
# define helpful functions for visualizing downloaded imagery
def show_rgb(img_file):
    with rio.open(img_file) as src:
        b,g,r,n = src.read()

    rgb = np.stack((r,g,b), axis=0)
    show(rgb/rgb.max())
    
def show_gray(img_file):
    with rio.open(img_file) as src:
        g = src.read(1)
    show(g/g.max())
    
# Replace this path with your image file path
img_file = out_folder+'2021-04-20_2021-08-25/PSScene4Band/20211007_202318_88_245c_3B_AnalyticMS_clip.tif'
show_rgb(img_file)

## You did it!

<div>
<img src="sandy-cheeks.jpeg" width="400"/>
</div>